In [1]:
#### import library for kinect
import cv2
from pykinect2 import PyKinectV2
from pykinect2 import PyKinectRuntime
import numpy as np
import datetime
import scipy.io 
import timeit

In [2]:
import argparse
import os
import sys
import time
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.utils.data
from sklearn.metrics import confusion_matrix
from data_loader_jpeg import VideoFolder
from model import ConvColumn
from torchvision.transforms import *
from PIL import Image
gpus = [0,0]
best_prec1 = int(0)


In [3]:
 # set run output folder
model_name = "jester_conv_example"
start = time.time()
# create model
model = ConvColumn(8)

# multi GPU setting
model = torch.nn.DataParallel(model, device_ids= gpus).cuda()

if os.path.isfile("checkpoint.pth.tar"):
    print("=> loading checkpoint")
    checkpoint = torch.load("checkpoint.pth.tar")
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint (epoch {})"
          .format(checkpoint['epoch']))
else:
    print("=> no checkpoint found ")

print(time.time() - start)
gestures = ['Swiping_Left', 'Swiping_Right', 'Swiping_Down', 'Swiping_Up', 'Zooming_In', 'Zooming_Out','Pointing', 'Doing_other_things']
colors = [(255,0,0),(0,255,0),(0,0,255),(0,255,255),(255,255,0),(255,0,255),(255,255,255)]

=> loading checkpoint
=> loaded checkpoint (epoch 46)
2.6009554862976074


In [ ]:
c = 'Doing_other_things'

class GestureBrain():
    
    def __init__(self,  model ):
        self.curr_stream = []
        self.model = model 
        self.counter  = 0 
        self.gt = ['Doing_other_things']
        self.gt = self.gt * 7   ##  number of threshold
        self.b = ['Doing_other_things','Doing_other_things']
        self.c = 'Doing_other_things'


    def adjust_judge_sequence(self):
        frame_diff = len(self.curr_stream) - 18 
        if  frame_diff == 0 :
            return self.curr_stream
        elif frame_diff > 0 :
            return self.curr_stream[frame_diff:]
        else :
            return self.curr_stream[:1] * abs(frame_diff) + self.curr_stream

    def prepreocess_img (self,  img_array):
        
#        img_array = (img_array / 255.)
#         img_array  = cv2.resize(img_array ,(149,84), interpolation = cv2.INTER_CUBIC)
        img_array = cv2.resize(img_array, (149, 84))
#        img_array = np.array(img_array, dtype=np.float32)
        img_array = Image.fromarray(img_array.astype('uint8'), 'RGB')
        transform = Compose([
        CenterCrop(84),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225])
        ])
        
        img_array = transform(img_array)
        return (img_array)
    

    def push_img (self, img):

        img = self.prepreocess_img(img )
        self.curr_stream.append(torch.unsqueeze(img, 0))
        if len( self.curr_stream ) > 18 :      ## limit 18 frames
            self.curr_stream.pop(0)

            
    def regonize(self):
        
        self.curr_stream = self.adjust_judge_sequence()
        data = torch.cat(self.curr_stream)
        data = data.permute(1, 0, 2, 3)
        data = data.unsqueeze(0)
        input = data
        
        input_vars = Variable(input)
        output = model(input_vars)
        _, predicted = torch.max(output.data, 1)
        predicted = predicted.cpu().numpy()
        predicted = predicted[0]
        gestures = ['Swiping_Left', 'Swiping_Right', 'Swiping_Down', 'Swiping_Up', 'Zooming_In', 'Zooming_Out','Pointing', 'Doing_other_things']
        print('predict_gesture: {}'.format(gestures[predicted]))
        index = predicted      
        return index 

    def gestures_opt(self, act):

        self.gt.append(act)
        self.gt.pop(0)

            
        if (len(set(self.gt))==1):        
            self.b.append(self.gt[-1])
        else:
            self.b.append('Doing_other_things')
            
        self.b.pop(0)
        if self.b[-1] != self.b[-2]:
            c = self.b[-1]
        else:
            c = 'Doing_other_things'
        print(c)
        return c
        


In [ ]:
# file = open("prediction_file.txt","w")
# file.close()  

# file = open("prediction_file1.txt","w")
# file.close()  

# get frame by  Kinect
kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Body)
print(type(kinect))

# Get one color picture frame
cv2.namedWindow('Color Image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Color Image', int(1920/2), int(1080/2))

frame = None
cnt = 0
x0 = 500
y0 = 500
h = 490/2
w = 650/2
blue_color = (255,0,0)

def draw_rectangle(img, x0, w, y1, y2, ay, color):
    # ok, at least one is good 
#     w = 406
#     h = 406
    s1 = (int(x0 - w),int(max(0, y1-ay)))      #### note: may be  transfer x to y
    s2 = (int(x0 + w),int(max(0, y1-ay)))
    s3 = (int(x0 + w),int(min(1080,y2)))
    s4 = (int(x0 - w),int(min(1080,y2)))
    cv2.line(img,s1,s2,color,5)
    cv2.line(img,s2,s3,color,5)
    cv2.line(img,s3,s4,color,5)
    cv2.line(img,s4,s1,color,5)




gb0 = GestureBrain(model)
gb1 = GestureBrain(model)
gb2 = GestureBrain(model)
gb3 = GestureBrain(model)
gb4 = GestureBrain(model)
gb5 = GestureBrain(model)
gb = [gb0,gb1,gb2,gb3,gb4,gb5]


counter = 0
index = [7]*5  # number  frames has hand over head    
inde = [0]* 50 # number frames has hand  below spine base

flat = 0    ## ( 0 = action recogtion, 1 = pointing ) 

while(True):

#     colorFrame = kinect.get_last_color_frame()
#     colorFrame = colorFrame.reshape((1080,1920,4))
#     cv2.imshow("Color Image", colorFrame)
    start = timeit.default_timer()  
    # gb = GestureBrain(model)
    
    if (kinect.has_new_body_frame()): 
        bodies = kinect.get_last_body_frame()
        if bodies is not None: 
            colorFrame = kinect.get_last_color_frame() 
            
            
            for i in range(0, kinect.max_body_count):
                body = bodies.bodies[i]      
                if not body.is_tracked: 
                    continue
                joints = body.joints
                # convert joint coordinates to color space 
                joint_points = kinect.body_joints_to_color_space(joints)     ### joint in color spaces
                joint0 = PyKinectV2.JointType_SpineShoulder
                joint1 = PyKinectV2.JointType_ShoulderLeft
                joint2 = PyKinectV2.JointType_ShoulderRight
                joint3 = PyKinectV2.JointType_Head
                joint4 = PyKinectV2.JointType_SpineBase   
                joint5 = PyKinectV2.JointType_SpineMid
                joint6 = PyKinectV2.JointType_Neck
                joint7 = PyKinectV2.JointType_HandTipLeft
                joint8 = PyKinectV2.JointType_HandTipRight
                
                
                x0 = joint_points[joint0].x
                y0 = joint_points[joint0].y  
                x1 = joint_points[joint1].x
                x2 = joint_points[joint2].x  
#                 w1 = abs(x2-x1)
                w1 = abs(joint_points[joint3].y - joint_points[joint5].y)  
    
                y1 = joint_points[joint3].y   
                y2 = (joint_points[joint4].y + joint_points[joint5].y)/2  
                y3 = joint_points[joint6].y  
                ay = abs(y3-y1)
                h1 = abs(y2-y1)                
                y_th_a = joint_points[joint3].y - abs(joint_points[joint6].y - joint_points[joint3].y)
#                 y_th_b = joint_points[joint4].y + abs(joint_points[joint4].y - joint_points[joint5].y)
                y_th_b = joint_points[joint4].y                
                
                
                
                #### convert to RGB frames
#               colorFrame = kinect.get_last_color_frame()
                colorFrame = colorFrame.reshape((1080,1920,4))
                rgb_frame = colorFrame[:,:,:-1]

        
        
                #### detect target person target_id                
                if joint_points[joint7].y < y_th_a or joint_points[joint8].y < y_th_a: 
                    index.append(i)
                    index.pop(0)
                    
#                 if index[-1] == index[-2]== index[-3]== index[-4]== index[-5]: 
                if(len(set(index))==1):         
                    target_id = index[-1] 
                if i == target_id: 
                    color = (0, 0, 255 )
                else:
                    color = (192,192,192)
                
                ######## draw block
                font = cv2.FONT_HERSHEY_SIMPLEX
                draw_rectangle(colorFrame, x0, w1, y1, y2, ay, color)
                cv2.putText(colorFrame, 'User_Id: {}'.format(i),(int(x0 - w1),int(y2 - 20)), font, 2,(0,255,0), 2, cv2.LINE_AA)
                   
                
#                 ########### boclk frame (note: may be change x,y)
#                 h1 = 480/2
#                 w1 = 640/2



################################################ Gesture Recognition ################################### 
                if flat == 0:
    
                    if i == target_id:             
                        for ind in range(0, 6):
                            if ind != target_id:
                                gb[ind].curr_stream = []
                        roi_frame = rgb_frame[int(max(0, y1-ay-20)):int(min(1080,y2 -20)),int(max(0,(x0 - w1))):int(min(1920,(x0 + w1))),:]
        #                     print(np.shape(roi_frame))

                        counter += 1
        #                 time.sleep(0.05)


                        gb[i].push_img(roi_frame)


                        action = gb[i].regonize()
                        act_name = gestures[action]
                        predict_gesture = gb[i].gestures_opt(act_name)


        #                 file = open("prediction_file1.txt", "a")
        #                 file.writelines("%s \n" % (predict_gesture))
        #                 file.close()

        #                 file = open("prediction_file.txt", "a")
        #                 file.writelines("%s \n" % (gestures[action]))
        #                 file.close()              


        #                 cv2.putText(colorFrame,'{}'.format(gestures[action]),(int(x0 - w),int(y0 - h)), font, 2 ,(0,255,0),2,cv2.LINE_AA)
        #                cv2.putText(colorFrame,'predict_gesture_per_frame: {}'.format(gestures[action]),(int(80),int(80)), font, 2 ,(255,0,0),2,cv2.LINE_AA)   

                        cv2.putText(colorFrame,'Gestures Recognition Module',(int(500),int(80)), font, 3 ,(255, 0, 0),3,cv2.LINE_AA)
                        if predict_gesture != 'Doing_other_things': 
                            cv2.putText(colorFrame,'{}'.format(predict_gesture),(int(500),int(300)), font, 5 ,color,5,cv2.LINE_AA)
                        cv2.imshow("Color Image", colorFrame)

                        if predict_gesture != 'Doing_other_things': 
                            key = cv2.waitKey(450)
                            time.sleep(0.6)
                            gb[i].curr_stream = []

                        if predict_gesture == 'Pointing': 
                            flat = 1

                    else: 
                        cv2.imshow("Color Image", colorFrame)
                    
######################################## Finger pointing ############################

                else: 
                    cv2.putText(colorFrame,'Mouse Cursor Module',(int(500),int(80)), font, 3 ,(255,0,0),3,cv2.LINE_AA)
                    if i == target_id: 
                        if joint_points[joint7].y < joint_points[joint4].y: 
                            cv2.circle(colorFrame,(int(joint_points[joint7].x),int(joint_points[joint7].y)),50, (0,255,0), -1)
                        if joint_points[joint8].y < joint_points[joint4].y: 
                            cv2.circle(colorFrame,(int(joint_points[joint8].x),int(joint_points[joint8].y)),50, (0,255,0), -1)                    
                        cv2.imshow("Color Image", colorFrame)
                        ##### check frame  hand below spinebase:  
                        if joint_points[joint7].y > y_th_b and joint_points[joint8].y > y_th_b: 
                            inde.append(0) 
                        else:
                            inde.append(1) 
                        inde.pop(0)    
    
                        if(len(set(inde))==1) and (inde[-1] == 0):
                            flat = 0
                            target_id = 7  
                            index = [7]*5
                pass
 
    key = cv2.waitKey(20)
    if  key == 27:
        break
    stop = timeit.default_timer()
    fps = 1/(stop-start)
    
    print("Estimated fps : {0}".format(fps))
cv2.destroyAllWindows()
kinect.close()

<class 'pykinect2.PyKinectRuntime.PyKinectRuntime'>
Estimated fps : 24.090692304002523
Estimated fps : 18.783068867874125
Estimated fps : 41.85220414042692
Estimated fps : 29.52075741036192
Estimated fps : 37.031157504450306
Estimated fps : 33.088565539440346
Estimated fps : 34.23900700240556
Estimated fps : 33.06833530861408
Estimated fps : 31.899057262886625
Estimated fps : 32.01058938148179
Estimated fps : 35.204852696721495
Estimated fps : 32.159635192740424
Estimated fps : 30.16385101800928
Estimated fps : 36.75511761630953
Estimated fps : 33.08700848909224
Estimated fps : 32.04969323475525
Estimated fps : 33.72951165691267
Estimated fps : 31.091657601726375
Estimated fps : 32.069866087717955
Estimated fps : 39.007045535238724
Estimated fps : 32.53372632124448
Estimated fps : 31.40270470643926
Estimated fps : 32.07191402793362
Estimated fps : 36.135543221297105
Estimated fps : 30.134892810916963
Estimated fps : 32.07513274820716
Estimated fps : 34.92888297185325
Estimated fps : 28